In [1]:
from tqdm import tqdm
from json import loads

In [18]:
minimum_matches = 50
n_worsts = 50
raw_data = "./raw_matches_2024_0710.csv"
output_dir = "./study"
date = "20241120"
chunk_size = 10000

In [19]:
dfs = pd.read_csv(raw_data, chunksize=chunk_size)

In [20]:
data = []
for df in dfs:
    for match in tqdm(df.match_data):
        info = loads(match)["info"]
        for part in info["participants"]:
            data.append(
                [
                    info["gameCreation"],
                    info["gameDuration"],
                    part["deaths"],
                    part["summonerName"],
                    part["summonerId"],
                    part["win"],
                ]
            )
user_df = pd.DataFrame(data, columns=["gameCreation","duration", "death", "name", "id", "win"])
user_df.to_csv(f"{output_dir}/{date}-user.csv", index=False)

100%|██████████| 1996/1996 [00:02<00:00, 718.75it/s]


In [22]:
name_df = (
    user_df[["name", "id"]]
    .drop_duplicates(
        subset="id",
        keep="last",
    )
    .reset_index(drop=True)
)
name_map = name_df.set_index("id").to_dict(orient="dict")["name"]

In [46]:
%%time
user_head = user_df.head(10000)

CPU times: user 191 µs, sys: 6 µs, total: 197 µs
Wall time: 185 µs


In [50]:
user_df = user_df.sort_values(['id', 'gameCreation'])


In [ ]:
%%time
def calculate_streaks(group):
    group['streak'] = (group['win'] != group['win'].shift()).cumsum()  # 구간 나누기
    streaks = group.groupby('streak')['win'].agg(['count', 'first']).reset_index()
    streaks = streaks.rename(columns={'count': 'length', 'first': 'win_status'})
    streaks['type'] = streaks['win_status'].apply(lambda x: 'win' if x else 'lose')
    streaks['id'] = group['id'].iloc[0]
    return streaks[['id', 'type', 'length']]

streaks_df = user_df.groupby('id', group_keys=False).apply(calculate_streaks).reset_index(drop=True)


In [ ]:
1

In [ ]:
)